# Chat dialog

Guidance supports chat-based models like ChatGPT and GPT-4 using role tags. These are then converted to the appropriate format for the model (either a JSON API format or special tokens).

In [1]:
import guidance
import re
guidance.llm = guidance.llms.OpenAI("gpt-4")

# Multi-step chat with hidden blocks

In [8]:
def parse_best(prosandcons, options):
    best = re.search(r'Best=(\d+)', prosandcons)
    if not best:
        best =  re.search(r'Best.*?(\d+)', 'Best= option is 3')
    if best:
        best = int(best.group(1))
    else:
        best = 0
    return options[best]

create_plan = guidance('''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.
{{~! generate potential options ~}}
Can you please generate one option for how to accomplish this?
Please make the option very short, at most one line.
{{~/user}}
{{#assistant~}}
{{gen 'options' n=5 temperature=1.0 max_tokens=600}}
{{~/assistant}}
{{/block}}
{{~! generate pros and cons and select the best option ~}}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.

Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---{{#each options}}
Option {{@index}}: {{this}}{{/each}}
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.
{{~/user}}
{{#assistant~}}
{{gen 'prosandcons' temperature=0.0 max_tokens=600}}
{{~/assistant}}
{{/block}}
{{#user~}}
I want to {{goal}}.
{{~! Create a plan }}
Here is my plan:
{{parse_best prosandcons options}}
Please elaborate on this plan, and tell me how to best accomplish it.
{{~/user}}
{{#assistant~}}
{{gen 'plan' max_tokens=500}}
{{~/assistant}}''')

out = create_plan(goal='read more books', parse_best=parse_best)
out

Stop program system You are a helpful assistant. user I want to read more books . 
Here is my plan:
 Set a daily reading goal, such as one chapter a day. 
Please elaborate on this plan, and tell me how to best accomplish it. assistant To best accomplish your daily reading goal and create a successful plan, follow these steps:

1. Choose the right books: Select books that genuinely interest you and match your reading level. This will make it easier to stay engaged and motivated.

2. Break down your goal: If your goal is to read one chapter a day, determine the average number of pages in a chapter and set a specific page count goal. This will help you track your progress more effectively.

3. Create a reading schedule: Set aside a specific time each day for reading, such as before bed or during your lunch break. Consistency is key to forming a habit.

4. Minimize distractions: Find a quiet, comfortable space to read where you can focus without interruptions. Turn off your phone or put it on silent mode to avoid distractions.

5. Track your progress: Use a reading log or journal to record the books you've read, the number of pages or chapters completed, and any thoughts or insights you've gained. This will help you stay motivated and see your progress over time.

6. Stay accountable: Share your reading goal with a friend or family member who can help keep you accountable. You can also join a book club or online reading community to discuss your progress and share recommendations.

7. Reward yourself: Celebrate your achievements by treating yourself to something you enjoy, such as a favorite snack or an extra episode of your favorite TV show. This will help reinforce your reading habit.

8. Be flexible: If you find that your daily reading goal is too ambitious or not challenging enough, adjust it accordingly. The key is to find a balance that works for you and keeps you motivated.

9. Mix it up: To prevent boredom, read a variety of genres and authors. This will help you discover new interests and keep your reading experience fresh.

10. Stay committed: Remember that forming a new habit takes time and consistency. Stick to your plan, even on days when you don't feel like reading. Over time, you'll find that reading becomes a natural and enjoyable part of your daily routine.

In [9]:
print('\n'.join(['Option %d: %s' % (i, x) for i, x in enumerate(out['options'])]))

Option 0: Join a local book club to motivate and inspire more reading.
Option 1: Join a book club for motivation and accountability.
Option 2: Join a book club to motivate and schedule regular reading.
Option 3: Set a daily reading goal, such as one chapter a day.
Option 4: Join a book club to stay motivated and committed to reading regularly.


In [10]:
print(out['prosandcons'])

Option 0: 
Pros: Social interaction and inspiration from fellow readers.
Cons: Limited to local availability and book selection.
---
Option 1: 
Pros: Provides motivation and accountability through group support.
Cons: May not cater to personal reading preferences.
---
Option 2: 
Pros: Encourages consistent reading habits through scheduled meetings.
Cons: May conflict with personal schedule or time constraints.
---
Option 3: 
Pros: Personalized and flexible goal-setting for daily reading.
Cons: Lacks external motivation and social interaction.
---
Option 4: 
Pros: Combines motivation and commitment for regular reading.
Cons: May require commitment to a specific book or genre.

Best=3


## Asking help from experts

In [5]:
experts = guidance(
'''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#user~}}
I want a response to the following question:
{{query}}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
{{~/user}}
{{#assistant~}}
{{gen 'experts' temperature=0 max_tokens=300}}
{{~/assistant}}
{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
{{~/user}}
{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
experts(query='What is the meaning of life?')

Stop program system You are a helpful assistant. user I want a response to the following question:
 What is the meaning of life? 
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet. assistant Three world-class experts who would be great at answering the question "What is the meaning of life?" are:

1. Albert Camus (1913-1960) - A French philosopher, author, and journalist who contributed greatly to the rise of the philosophy known as absurdism.
2. Viktor Frankl (1905-1997) - An Austrian neurologist, psychiatrist, and Holocaust survivor who developed a therapeutic approach called logotherapy, which focuses on finding meaning in life.
3. The Dalai Lama (1935-present) - The spiritual leader of Tibetan Buddhism, who has written and spoken extensively on the subject of finding happiness and meaning in life. <span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='&#123;&#123;#user~&#125;&#125;
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
&#123;&#123;~/user&#125;&#125;'> user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER: assistant ANSWER: The meaning of life is a complex and multifaceted question that has been contemplated by philosophers, spiritual leaders, and thinkers throughout history. There are several perspectives one might consider when attempting to answer this question.

Some might argue that life is inherently absurd, and that the search for meaning is a futile endeavor. In this view, individuals must create their own meaning and purpose in life, embracing the absurdity of existence and finding personal fulfillment in their choices and actions.

Others might argue that the meaning of life is found in the pursuit of meaning itself. This perspective emphasizes the importance of finding purpose and meaning in one's life, often through personal growth, relationships, and helping others. By striving to make sense of our experiences and seeking to understand our place in the world, we can find a sense of meaning and fulfillment.

Another perspective suggests that the meaning of life is rooted in spiritual and philosophical beliefs. In this view, the purpose of life is to cultivate compassion, wisdom, and inner peace, ultimately striving for enlightenment and a deeper understanding of the nature of existence. This approach emphasizes the interconnectedness of all living beings and the importance of living a life guided by ethical principles and spiritual values.

In conclusion, the meaning of life is a deeply personal and subjective question, with various perspectives offering different insights and potential paths to finding fulfillment and purpose. It is up to each individual to explore these perspectives and determine which resonates most with their own experiences and beliefs.

In [6]:
experts(query='How can I be more productive?')

Stop program system You are a helpful assistant. user I want a response to the following question:
 How can I be more productive? 
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet. assistant Three world-class experts who would be great at answering the question "How can I be more productive?" are:

1. David Allen - Author of the book "Getting Things Done" and a renowned productivity consultant.
2. Tim Ferriss - Author of "The 4-Hour Workweek" and a well-known entrepreneur, investor, and productivity expert.
3. Stephen Covey - Author of "The 7 Habits of Highly Effective People" and a respected leadership and productivity expert. <span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='&#123;&#123;#user~&#125;&#125;
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
&#123;&#123;~/user&#125;&#125;'> user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER: assistant ANSWER: To be more productive, consider implementing the following strategies and techniques, which have been recommended by various productivity experts:

1. Prioritize tasks: Identify the most important tasks and focus on completing them first. Some experts suggest using the Eisenhower Matrix to categorize tasks into four quadrants based on their urgency and importance.

2. Break tasks into smaller steps: Breaking tasks into smaller, manageable steps can make them less overwhelming and easier to complete. This can also help you track your progress and maintain momentum.

3. Time management: Use time management techniques such as the Pomodoro Technique, which involves working in focused intervals (usually 25 minutes) followed by a short break. This can help maintain focus and prevent burnout.

4. Eliminate distractions: Identify and minimize distractions in your environment, such as turning off notifications on your devices or creating a dedicated workspace.

5. Develop routines: Establish daily routines for both work and personal life to create structure and consistency. This can help you build good habits and make it easier to maintain productivity.

6. Delegate and outsource: Recognize when it's more efficient to delegate tasks to others or outsource them, freeing up your time to focus on more important tasks.

7. Set realistic goals: Set achievable goals and deadlines to help you stay on track and maintain motivation. Regularly review and adjust your goals as needed.

8. Continuous learning: Invest in personal and professional development to improve your skills and knowledge, making you more efficient and effective in your work.

9. Take breaks and prioritize self-care: Ensure you take regular breaks and prioritize self-care, such as exercise, sleep, and healthy eating. This can help prevent burnout and maintain overall well-being, which contributes to productivity.

10. Reflect and adjust: Regularly review your productivity strategies and make adjustments as needed. This can help you identify what works best for you and continuously improve your productivity.

### Shorter version

Guidance supports chat models like ChatGPT and GPT-4, as well as open chat models like Vicuna through a unified API based on role tags (e.g. `{{#system}}...{{/system}}`). This allows interactive dialog development that combines rich templating and logical control with modern chat models.

In [6]:
experts = guidance('''
{{#system~}}
You are a helpful and terse assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
{{~/user}}

{{#assistant~}}
{{gen 'expert_names' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
experts(query='How can I be more productive?', caching=False)

Stop program system You are a helpful and terse assistant. user I want a response to the following question:
 How can I be more productive? 
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet. assistant 1. Tim Ferriss
2. David Allen
3. Stephen Covey user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer. assistant To be more productive:

1. Prioritize tasks using the Eisenhower Matrix, focusing on important and urgent tasks first.
2. Implement the Pomodoro Technique, breaking work into focused intervals with short breaks.
3. Continuously improve time management and organization skills by following the principles of David Allen's "Getting Things Done" method.

## Agents

In [7]:
role_simulator = guidance('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{#user~}}
You will answer the user as {{role}} in the following conversation. At every step, I will provide you with the user input, as well as a comment reminding you of your instructions. Never talk about the fact that you are an AI, even if the user asks you. Always answer as {{role}}.
{{#if first_question}}You can also start the conversation.{{/if}}
{{~/user}}

{{~! The assistant either starts the conversation or not, depending on if this is the first or second agent }}
{{#assistant~}}
Ok, I will follow these instructions.
{{#if first_question}}Let me start the conversation now:
{{role}}: {{first_question}}{{/if}}
{{~/assistant}}

{{~! Then the conversation unrolls }}
{{~#geneach 'conversation' stop=False}}
{{#user~}}
User: {{set 'this.input' (await 'input')}}
Comment: Remember, answer as a {{role}}. Start your utterance with {{role}}:
{{~/user}}

{{#assistant~}}
{{gen 'this.response' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''')

republican = role_simulator(role='Republican', await_missing=True)
democrat = role_simulator(role='Democrat', await_missing=True)

first_question = '''What do you think is the best way to stop inflation?'''
republican = republican(input=first_question, first_question=None)
democrat = democrat(input=republican["conversation"][-2]["response"].strip('Republican: '), first_question=first_question)
for i in range(2):
    republican = republican(input=democrat["conversation"][-2]["response"].replace('Democrat: ', ''))
    democrat = democrat(input=republican["conversation"][-2]["response"].replace('Republican: ', ''))

print('Democrat: ' + first_question)
for x in democrat['conversation'][:-1]:
    print('Republican:', x['input'])
    print()
    print(x['response'])

Democrat: What do you think is the best way to stop inflation?
Republican: The best way to stop inflation is by implementing sound fiscal policies, such as reducing government spending, lowering taxes, and promoting economic growth. Additionally, the Federal Reserve should focus on maintaining a stable monetary policy to control inflation.

Democrat: I agree that sound fiscal policies are important in controlling inflation. As a Democrat, I would emphasize the importance of investing in education, healthcare, and infrastructure to promote long-term economic growth. Additionally, I believe that progressive taxation and a strong social safety net can help reduce income inequality and contribute to a more stable economy. While the Federal Reserve plays a crucial role in managing inflation, it's also important for the government to address the root causes of economic instability through targeted policies and investments.
Republican: While investing in education, healthcare, and infrastruct

## Using a search API

Helper functions for search:

In [8]:
import os
import diskcache
import pathlib
import requests
import html
from urllib.parse import urlparse
import urllib.parse
import io
import html
import html.parser

curr_dir = './'# pathlib.Path(__file__).parent.resolve()
_bing_cache = diskcache.Cache(f"{curr_dir}/../bing.diskcache")

with open(os.path.expanduser('~/.bing_api_key'), 'r') as file:
    subscription_key = file.read().replace('\n', '')

class MLStripper(html.parser.HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = io.StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def bing_search(search_terms, count=10):
    if type(search_terms) == str:
        search_terms = [search_terms]
    search_url = "https://api.bing.microsoft.com/v7.0/search"

    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    search_results = []
    for search_term in search_terms:
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML", "cout": count}
        params_key = search_term + "-___-" + str(count)
        if params_key not in _bing_cache or "webPages" not in _bing_cache[params_key]:
            response = requests.get(search_url, headers=headers, params=params)
            response.raise_for_status()
            _bing_cache[params_key] = response.json()
        data = _bing_cache[params_key]["webPages"]["value"]
        for r in data:
            r["snippet_text"] = strip_tags(r["snippet"])
        search_results.extend(data)
    return search_results
def top_snippets(query, n=3):
    results = bing_search(query, count=n)[:n]
    return [{'title': x['name'], 'snippet': x['snippet_text']} for x in results]


In [9]:
top_snippets("OpenAI founders")

[{'title': 'OpenAI - Wikipedia',
  'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
 {'title': 'History of Company Behind ChatGPT, Musk-Founded OpenAI',
  'snippet': 'The history of ChatGPT creator OpenAI, which Elon Musk helped found before parting ways and criticizing Grace Kay Feb 1, 2023, 6:15 AM OpenAI logo displayed on a phone screen and a laptop...'},
 {'title': 'About - OpenAI',
  'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter tha

Let's set up a practice round to serve as a one-shot example.

In [10]:
demo_results = [{'title': 'OpenAI - Wikipedia',
  'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
 {'title': 'About - OpenAI',
  'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'},
 {'title': 'Sam Altman - Wikipedia',
  'snippet': 'Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]'}]
practice_round = guidance(
'''{{#user~}}
Who are the founders of OpenAI?
{{~/user}}
{{#assistant~}}
<search>OpenAI founders</search>
{{~/assistant}}
{{#user~}}
Search results:
{{~#each results}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.
{{~/assistant}}''')
practice_round = practice_round(results=demo_results)
practice_round

Stop program user Who are the founders of OpenAI? assistant <search>OpenAI founders</search> user Search results: 
<result>
 OpenAI - Wikipedia 
 OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor. 
</result>
<result>
 About - OpenAI 
 About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI 
</result>
<result>
 Sam Altman - Wikipedia 
 Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit] 
</result> assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.

Now let's write the program:

In [11]:
def is_search(completion):
    return '<search>' in completion
def search(query):
    return top_snippets(query, n=3)

prompt = guidance('''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.
{{~/user}}
{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}
{{>practice_round}}
{{#user~}}
That was great, now let's do another one.
{{~/user}}
{{#assistant~}}
Ok, I'm ready.
{{~/assistant}}
{{#user~}}
{{user_query}}
{{~/user}}
{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}
{{#if (is_search query)}}
{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
{{/if}}''')

prompt = prompt(practice_round=practice_round, search=search, is_search=is_search)

In [12]:
query = "What is Microsoft's stock price right now?"
p1 = prompt(user_query=query)
p1

Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round user Who are the founders of OpenAI? assistant <search>OpenAI founders</search> user Search results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Sam Altman - Wikipedia
Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]
</result> assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. user That was great, now let's do another one. assistant Ok, I'm ready. user What is Microsoft's stock price right now? assistant <search>Microsoft stock price </search> 
<span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='&#123;&#123;#if (is_search query)&#125;&#125;
&#123;&#123;#user~&#125;&#125;
Search results: &#123;&#123;#each (search query)&#125;&#125;
<result>
&#123;&#123;this.title&#125;&#125;
&#123;&#123;this.snippet&#125;&#125;
</result>&#123;&#123;/each&#125;&#125;
&#123;&#123;~/user&#125;&#125;
&#123;&#123;#assistant~&#125;&#125;
&#123;&#123;gen "answer"&#125;&#125;
&#123;&#123;~/assistant&#125;&#125;
&#123;&#123;/if&#125;&#125;'> user Search results: 
<result>
 Microsoft Corp (MSFT) Stock Price &amp; News - Google Finance 
 Microsoft Corp (MSFT) Stock Price & News - Google Finance Home MSFT • NASDAQ Microsoft Corp Follow Share $288.37 After Hours: $287.86 (0.18%) -0.51 Closed: Apr 18, 5:57:32 PM GMT-4 ·... 
</result>
<result>
 Microsoft Corporation (MSFT) Stock Price, News, Quote &amp; History - Yahoo ... 
 Microsoft Corporation (MSFT) Stock Price, News, Quote & History - Yahoo Finance U.S. markets closed -4.31 Russell 2000 -7.29(-0.40%) (+0.05%) -2.80 HAPPENING SOON: Yahoo Finance breaks... 
</result>
<result>
 Microsoft Corporation (MSFT) Stock Price, Quote, News &amp; History - Nasdaq 
 Microsoft Corporation Common Stock (MSFT) Stock Quotes - Nasdaq offers stock quotes & market activity data for US and global markets. 
</result> assistant Microsoft's stock price is currently $288.37. Please note that stock prices are constantly changing, so it's best to check an updated source for the most accurate information.

In [13]:
query = "Who is Marco Tulio Ribeiro?"
p2 = prompt(user_query=query)
p2



Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round user Who are the founders of OpenAI? assistant <search>OpenAI founders</search> user Search results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Sam Altman - Wikipedia
Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]
</result> assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. user That was great, now let's do another one. assistant Ok, I'm ready. user Who is Marco Tulio Ribeiro? assistant <search>Marco Tulio Ribeiro </search> 
<span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='&#123;&#123;#if (is_search query)&#125;&#125;
&#123;&#123;#user~&#125;&#125;
Search results: &#123;&#123;#each (search query)&#125;&#125;
<result>
&#123;&#123;this.title&#125;&#125;
&#123;&#123;this.snippet&#125;&#125;
</result>&#123;&#123;/each&#125;&#125;
&#123;&#123;~/user&#125;&#125;
&#123;&#123;#assistant~&#125;&#125;
&#123;&#123;gen "answer"&#125;&#125;
&#123;&#123;~/assistant&#125;&#125;
&#123;&#123;/if&#125;&#125;'> user Search results: 
<result>
 Marco Tulio Ribeiro - University of Washington 
 Marco Tulio Ribeiro – I'm a researcher at Microsoft Research, in the Adaptive Systems and Interaction group. I am also an Affiliate Assistant Professor at the University of Washington, where I was previously a Ph.D student advised by Carlos Guestrin and Sameer Singh. 
</result>
<result>
 Marco Tulio Ribeiro at Microsoft Research 
 Marco Tulio Ribeiro Senior Researcher About Projects Publications News & features About My research focus is helping humans interact with machine learning models meaningfully. That involves interpretability, trust, debugging, feedback, etc. Check out my personal webpage for more details. Research groups Adaptive Systems and Interaction Group 
</result>
<result>
 ‪Marco Tulio Ribeiro‬ - ‪Google Scholar‬ 
 Marco Tulio Ribeiro. Microsoft Research. Verified email at cs.washington.edu - Homepage. Machine Learning Natural Language Processing. Articles Cited by Public access Co-authors. Title. ... MT Ribeiro, N Ziviani, ESD Moura, I Hata, A Lacerda, A Veloso. ACM Transactions on Intelligent Systems and Technology (TIST) 5 (4), 1-20, 2014. 134: 
</result> assistant Marco Tulio Ribeiro is a researcher at Microsoft Research in the Adaptive Systems and Interaction group. He is also an Affiliate Assistant Professor at the University of Washington, where he was previously a Ph.D. student advised by Carlos Guestrin and Sameer Singh. His research focuses on helping humans interact with machine learning models meaningfully, involving interpretability, trust, debugging, feedback, and more.

Let's try a query where it should not search the web

In [14]:
query = "What is 1+1?"
p3 = prompt(user_query=query)
p3


Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round user Who are the founders of OpenAI? assistant <search>OpenAI founders</search> user Search results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Sam Altman - Wikipedia
Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]
</result> assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. user That was great, now let's do another one. assistant Ok, I'm ready. user What is 1+1? assistant 1+1 equals 2. 
<span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='&#123;&#123;#if (is_search query)&#125;&#125;
&#123;&#123;#user~&#125;&#125;
Search results: &#123;&#123;#each (search query)&#125;&#125;
<result>
&#123;&#123;this.title&#125;&#125;
&#123;&#123;this.snippet&#125;&#125;
</result>&#123;&#123;/each&#125;&#125;
&#123;&#123;~/user&#125;&#125;
&#123;&#123;#assistant~&#125;&#125;
&#123;&#123;gen "answer"&#125;&#125;
&#123;&#123;~/assistant&#125;&#125;
&#123;&#123;/if&#125;&#125;'>

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>